# Pré-processamento dos Dados
## | Avaliando Riscos do Modelo e Selecionando Variáveis Explicativas

### > **Objetivo do Notebook**: 

Este notebook é a 2ª parte do desenvolvimento do projeto focado para construção de um modelo preditivo para classificar alunos com alta chance de evasão do ensino superior. O foco desse notebook será em avaliar os riscos do modelo, dado a abordagem que será usada para a modelagem e selecionar as variáveis explicativas de maior importância através de parâmetros como **Information Value (IV)**, **Variance Inflation Factor (VIF)** e **Z-score**.

### > **Insights da Exploração**

Com base no jupyter notebook anterior, já possuímos uma base com qual podemos começar o trabalho de avaliar as variáveis explicativas. Para avaliar essas variáveis pré-selecionadas tendo em vista o modelo de **Regressão Logística** a ser construído, vamos avaliar:

- Informação das Variáveis, calculado com **Information Value (IV)** baseado em **Weight of Evidence (WOE)** das classes para variáveis categóricas e continuas categorizadas
- Multicolinearidade, calculado com **Variance Inflation Factor (VIF)**
- Outliers, calculado com **Z-score**
  
Conforme o desenrolar da análise, vamos transformar as variáveis de acordo com as necessidades de obter os melhores indicadores das variáveis.

## | Importando Bibliotecas

In [8]:
# Manipulação e transformação de dados
import numpy as np
import pandas as pd

# Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# Dividir base de dados
from sklearn.model_selection import train_test_split

# Calculos estatísticos
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

# Ferramentas de análise Z-score
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## | Extraindo e Preparando Dados

In [9]:
# Listando variáveis de interesse
vars = [
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)',
    'Tuition fees up to date',
    'Scholarship holder',
    'Course',
    'Application mode',
    'Target'
]

# Extraindo dados
data = pd.read_csv('../data/raw/train.csv')

# Criando subconjunto selecionando apenas variáveis de interesse
df = data[vars].copy()

# Mapeando Target para estudar apenas o caso de evasão de alunos
df['Target'] = df['Target'].map({'Graduate': 0, 'Enrolled': 0, 'Dropout': 1})

# Dividindo variáveis pelo tipo
vars_continuas = vars[:4]
vars_binarias = ['Tuition fees up to date', 'Scholarship holder', 'Target']
vars_nominais = ['Course', 'Application mode']

# Criando mapas para mapear variáveis categóricas nominais
cursos = {
    33: "Tecnologias de Produção de Biocombustíveis",
    171: "Animação e Design Multimédia",
    8014: "Serviço Social (atendimento noturno)",
    9003: "Agronomia",
    9070: "Design de Comunicação",
    9085: "Enfermagem Veterinária",
    9119: "Engenharia Informática",
    9130: "Equinicultura",
    9147: "Gestão",
    9238: "Serviço Social",
    9254: "Turismo",
    9500: "Enfermagem",
    9556: "Higiene Oral",
    9670: "Gestão de Publicidade e Marketing",
    9773: "Jornalismo e Comunicação",
    9853: "Ensino Básico",
    9991: "Gestão (atendimento noturno)"
}

aplicacoes = {
    1: "1ª fase – contingente geral",
    2: "Portaria nº 612/93",
    5: "1ª fase - contingente especial (Ilha dos Açores)",
    7: "Titulares de outros cursos superiores",
    10: "Portaria nº 854-B/99",
    15: "Estudante internacional (bacharelado)",
    16: "1ª fase – contingente especial (Ilha da Madeira)",
    17: "2ª fase – contingente geral",
    18: "3ª fase – contingente geral",
    26: "Portaria n.º 533-A/99, alínea b2) (Plano Diferente)",
    27: "Portaria nº 533-A/99, item b3 (Outra Instituição)",
    39: "Maiores de 23 anos",
    42: "Transferência",
    43: "Mudança de curso",
    44: "Titulares de diploma de especialização tecnológica",
    51: "Mudança de instituição/curso",
    53: "Titulares de diplomas de ciclo curto",
    57: "Mudança de instituição/curso (Internacional)"
}

# Mapeando variáveis nominais
df['Course'] = df['Course'].map(cursos)
df['Application mode'] = df['Application mode'].map(aplicacoes)

# Visualizando
df.head(5)

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Course,Application mode,Target
0,6,14.500000,6,12.428571,1,1,Serviço Social,1ª fase – contingente geral,0
1,4,11.600000,0,0.000000,1,0,Serviço Social,2ª fase – contingente geral,1
2,0,0.000000,0,0.000000,1,0,Turismo,2ª fase – contingente geral,1
3,7,12.591250,7,12.820000,1,1,Enfermagem,1ª fase – contingente geral,0
4,6,12.933333,6,12.933333,1,0,Enfermagem,1ª fase – contingente geral,0


In [10]:
print('Numero de linhas: {0} \nNúmero de colunas: {1}'.format(df.shape[0], df.shape[1]))

Numero de linhas: 76518 
Número de colunas: 9


Com base na análise do notebook anterior, vamos focar apenas nas variáveis que descrevem o desempenho acadêmico do aluno ao longo do ano, além das variáveis relacionadas à condição financeira, tipo de aplicação e curso.

In [11]:
# Carregado informações sobre variáveis do notebook EDA
info = pd.read_feather('../data/processed/classificacao_variaveis_categoricas.feather')

# Selecionando apenas sobre variáveis de interesse
info = info[info['Variável'].isin(vars)]

# Visualizando
info

,Variável,Classificação
1,Application mode,Altamente desbalanceada
3,Course,Totalmente desbalanceada
14,Tuition fees up to date,Moderadamente desbalanceada
16,Scholarship holder,Balanceada
18,Target,Altamente desbalanceada


Vamos reclassificar as variáveis com base na nova distribuição da variável `Target`, garantindo uma categorização mais alinhada com a estrutura dos dados.

Além disso, calcularemos o **Information Value (IV)** para medir a relevância de cada variável na previsão da variável alvo e o **Variance Inflation Factor (VIF)** para detectar possíveis problemas de multicolinearidade. Essas métricas serão analisadas tanto antes quanto depois das transformações, permitindo uma avaliação completa da importância das variáveis e da redundância entre elas.

Com essa abordagem, garantiremos um modelo mais eficiente, interpretável e robusto, otimizando a seleção das variáveis mais relevantes para a modelagem.

## | Avaliando Dados

In [12]:
# Função para calcular o índice de gini 
def calcular_gini(pi):
    return 1 - np.sum(pi**2)

# Função para calcular o índice de shannon 
def calcular_shannon(pi):
    return -np.sum(pi * np.log(pi))

# Função para classificar variável
def classificar_variavel(variavel: pd.Series) -> str:
    '''
    Função para classificar a variável com base no seu cálculo de Gini e Shannon

    Parâmetros:
    - variavel: uma Serie do Pandas, sendo essa a variável a se classificar

    Retorna:
    - string com a classificação da variável
    '''
    # Cálculos iniciais
    gini = calcular_gini(variavel.value_counts(normalize=True))
    shannon = calcular_shannon(variavel.value_counts(normalize=True))
    max_shannon = np.log(variavel.nunique())
    
    # Classificação pelo Gini
    if gini >= 0.8:
        classe_gini = 'Totalmente desbalanceada'
    elif gini >= 0.6:
        classe_gini = 'Altamente desbalanceada'
    elif gini >= 0.4:
        classe_gini = 'Moderadamente desbalanceada'
    else:
        classe_gini = 'Balanceada'
    
    # Classificação pelo Shannon
    if shannon <= 0.2 * max_shannon:
        classe_shannon = 'Totalmente desbalanceada'
    elif shannon <= 0.4 * max_shannon:
        classe_shannon = 'Altamente desbalanceada'
    elif shannon <= 0.7 * max_shannon:
        classe_shannon = 'Moderadamente desbalanceada'
    else:
        classe_shannon = 'Balanceada'
    
    # Ajuste final da classificação considerando Gini e Shannon
    if classe_gini == 'Totalmente desbalanceada' or classe_shannon == 'Totalmente desbalanceada':
        classe_final = 'Totalmente desbalanceada'
    elif classe_gini == 'Altamente desbalanceada' or classe_shannon == 'Altamente desbalanceada':
        classe_final = 'Altamente desbalanceada'
    elif classe_gini == 'Moderadamente desbalanceada' or classe_shannon == 'Moderadamente desbalanceada':
        classe_final = 'Moderadamente desbalanceada'
    else:
        classe_final = 'Balanceada'

    return classe_final

for var in info['Variável']:
    print(f'{var}')
    print(f"Classificação Anterior: {info[info['Variável'] == var]['Classificação'].values[0]}")
    print(f'Classificação Nova: {classificar_variavel(df[var])}')
    print('='*50)

Application mode
Classificação Anterior: Altamente desbalanceada
Classificação Nova: Altamente desbalanceada
Course
Classificação Anterior: Totalmente desbalanceada
Classificação Nova: Totalmente desbalanceada
Tuition fees up to date
Classificação Anterior: Moderadamente desbalanceada
Classificação Nova: Moderadamente desbalanceada
Scholarship holder
Classificação Anterior: Balanceada
Classificação Nova: Balanceada
Target
Classificação Anterior: Altamente desbalanceada
Classificação Nova: Moderadamente desbalanceada


Até o momento, a única variável transformada foi a variável alvo, `Target`, que foi convertida em uma variável binária, resultando em um melhor balanceamento.

Com base nisso, aplicaremos **Padronização (Z-score)** para variáveis numéricas e **One-Hot Encoding** para variáveis categóricas, garantindo que todas estejam em um formato adequado para modelagem.

In [13]:
def categorizar_variavel(variavel: pd.Series, intervalos: int=10) -> pd.Series:
    '''
    Função para categorizar uma variável contínua.

    Parâmetros:
    - variavel: uma Series do Pandas, sendo essa a variável a ser categorizada.
    - intervalos: um número inteiro que representa o número de intervalos a criar para categorizar a variável.

    Retorna:
    - a Series Pandas da variável categorizada.
    '''
    labels = pd.qcut(variavel, intervalos, duplicates='drop')
    cat_variavel = pd.Series(labels, name=f'cat_{variavel.name}')
    return cat_variavel
    
def criar_tabela_woe(variavel: pd.Series, resposta: pd.Series) -> tuple:
    '''
    Função para construir a tabela cruzada e calcular o Weight of Evidence das classes e Information Value da variável.

    Parâmetros:
    - variavel: uma Series do Pandas, sendo essa a variável a ser estudada.
    - resposta: uma Series do Pandas, sendo essa a variável resposta binária a ser usada como base.

    Retorna:
    - tupla contendo a tabela cruzada e o cálculo de information value.
    '''
    # Verifica se a variável esta na lista de variáveis contínuas
    if variavel.name in vars_continuas:
        variavel = categorizar_variavel(variavel, intervalos=5)
        
    # Cria a tabela cruzada
    tab = pd.crosstab(variavel, resposta, margins=True)

    # Calculada as probabilidades do evento e não-evento
    tab['Prop_nao_evento'] = tab[0] / tab[0].sum()
    tab['Prop_evento'] = tab[1] / tab[1].sum()

    # Define um valor para impedir zeros estruturais no cálculo
    epsilon = 1e-10

    # Calcula Weight of Evidence e Information Value parcial das classes
    tab['WOE'] = np.log((tab['Prop_nao_evento'] + epsilon) / (tab['Prop_evento'] + epsilon))
    tab['IV_parcial'] = (tab['Prop_nao_evento'] - tab['Prop_evento']) * tab['WOE']

    # Calcula o Information Value da variável categórica
    iv = tab['IV_parcial'].sum()

    return iv, tab

def classificar_iv(iv: float) -> str:
    '''
    Função para classificar o IV de acordo com a classificação de Naeem Siddiqi.

    Parâmetros:
    - iv: o valor de Information Value da variável.

    Retorna:
    - string com a classificação de IV da variável.
    '''
    if iv < 0.02:
        return 'Inútil'
    elif 0.02 <= iv < 0.1:
        return 'Fraca'
    elif 0.1 <= iv < 0.3:
        return 'Média'
    elif 0.3 <= iv < 0.5:
        return 'Forte'
    return 'Sobreajuste'

def classificar_vif(vif: float) -> str:
    '''
    Função para classificar o VIF de acordo com a classificação mais usada de valores VIF

    Parâmetros:
    - vif: o valor de Variance Inflation Factor da variável

    Retorna:
    - string com a classificação de VIF da variável
    '''
    if vif < 5:
        return 'Baixa multicolinearidade'
    elif 5 <= vif < 10:
        return 'Moderada multicolinearidade'
    elif vif >= 10:
        return 'Alta multicolinearidade'
    else:
        return 'Não identificado'
        
def calcular_iv_vif(features: pd.DataFrame, label: pd.Series) -> pd.DataFrame:
    '''
    Função para calcular e classificar IV e VIF das variáveis.

    Parâmetros:
    - features: pandas DataFrame contendo as variáveis independentes (features)
    - label: pandas Series contendo a variável dependente (label)

    Retorna:
    - pandas DataFrame com os valores de IV e VIF calculados e suas classificações
    '''
    resultados = []
    
    # Calcula Information Value para cada feature
    for coluna in features.columns:
        iv, _ = criar_tabela_woe(features[coluna], label)
        iv_class = classificar_iv(iv)
        resultados.append({'Feature': coluna, 'IV': iv, 'IV_Classification': iv_class})
    
    # Converte variáveis categóricas para valores numéricos
    features_numeric = pd.get_dummies(features, drop_first=True).astype(np.int64)
    
    # Calcula Variance Inflation Factor 
    vif_data = features_numeric.copy()
    vif_data['Intercept'] = 1  
    vif_values = [(vif(vif_data.values, i), classificar_vif(vif(vif_data.values, i))) for i in range(vif_data.shape[1])]
    
    vif_df = pd.DataFrame(vif_values, columns=['VIF', 'VIF_Classification'], index=vif_data.columns)
    
    # Remove intercepto dos resultados
    vif_df = vif_df.drop(index='Intercept')  
    
    # Junta os resultados de IV e VIF em um DataFrame
    df_result = pd.DataFrame(resultados)
    df_result = df_result.merge(vif_df, left_on='Feature', right_index=True, how='left')

    return df_result

In [14]:
# Calculando IV e VIF das variáveis pré-transformação
calcular_iv_vif(df.drop(columns=['Target']), df.Target)

,Feature,IV,IV_Classification,VIF,VIF_Classification
0,Curricular units 1st sem (approved),1.080928,Sobreajuste,9.334478,Moderada multicolinearidade
1,Curricular units 1st sem (grade),0.830554,Sobreajuste,6.153556,Moderada multicolinearidade
2,Curricular units 2nd sem (approved),1.216103,Sobreajuste,10.314467,Alta multicolinearidade
3,Curricular units 2nd sem (grade),1.027323,Sobreajuste,7.430781,Moderada multicolinearidade
4,Tuition fees up to date,0.555496,Sobreajuste,1.227124,Baixa multicolinearidade
5,Scholarship holder,0.322887,Forte,1.191600,Baixa multicolinearidade
6,Course,0.269512,Média,NaN,NaN
7,Application mode,0.357714,Forte,NaN,NaN


Conforme observado, as variáveis acadêmicas apresentam um alto risco de sobreajuste, o que pode comprometer a capacidade de generalização do modelo final. Por outro lado, as demais variáveis demonstram um bom potencial para a modelagem.

Em relação à multicolinearidade, identificamos que a variável `Curricular units 2nd sem (approved)` apresenta um nível elevado, o que pode impactar negativamente a estabilidade do modelo. Além disso, não foi possível calcular o VIF para as variáveis categóricas `Course` e `Application mode`, pois elas precisam ser codificadas utilizando **One-Hot Encoding** antes da análise.

Na próxima etapa, aplicaremos as transformações necessárias nos dados e reavaliamos os parâmetros para entender como essas mudanças afetam a qualidade e a robustez do modelo.

## | Transformando Dados

In [15]:
# Transformando base
df_transformado = df.copy()

# Padronizando variáveis contínuas
scaler = StandardScaler()
df_transformado[vars_continuas] = scaler.fit_transform(df_transformado[vars_continuas])

# Aplicando One-hot encoding nas variáveis categóricas nominais
for var in vars_nominais:
    dummies = pd.get_dummies(df_transformado[var], drop_first=True, prefix=var).astype(np.int64)
    df_transformado = pd.concat([df_transformado.drop(columns=var), dummies], axis=1)

# Visualizando
df_transformado.head(5)

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Target,Course_Animação e Design Multimédia,Course_Design de Comunicação,Course_Enfermagem,...,Application mode_Mudança de curso,Application mode_Mudança de instituição/curso,"Application mode_Portaria n.º 533-A/99, alínea b2) (Plano Diferente)","Application mode_Portaria nº 533-A/99, item b3 (Outra Instituição)",Application mode_Portaria nº 612/93,Application mode_Portaria nº 854-B/99,Application mode_Titulares de diploma de especialização tecnológica,Application mode_Titulares de diplomas de ciclo curto,Application mode_Titulares de outros cursos superiores,Application mode_Transferência
0,0.677640,0.855619,0.718660,0.505317,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.066414,0.304727,-1.445110,-1.735681,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.554522,-1.898842,-1.445110,-1.735681,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.049667,0.493027,1.079288,0.575895,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.677640,0.558010,0.718660,0.596330,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [16]:
print('Numero de linhas: {0} \nNúmero de colunas: {1}'.format(df_transformado.shape[0], df_transformado.shape[1]))

Numero de linhas: 76518 
Número de colunas: 39


In [17]:
# Calculando IV e VIF das variáveis pós-transformação
resultado = calcular_iv_vif(df_transformado.drop(columns=['Target']), df_transformado.Target)
resultado

,Feature,IV,IV_Classification,VIF,VIF_Classification
0,Curricular units 1st sem (approved),1.080928e+00,Sobreajuste,6.408728,Moderada multicolinearidade
1,Curricular units 1st sem (grade),8.305542e-01,Sobreajuste,4.204378,Baixa multicolinearidade
2,Curricular units 2nd sem (approved),1.216103e+00,Sobreajuste,7.923894,Moderada multicolinearidade
3,Curricular units 2nd sem (grade),1.027323e+00,Sobreajuste,4.832967,Baixa multicolinearidade
4,Tuition fees up to date,5.554957e-01,Sobreajuste,1.213432,Baixa multicolinearidade
5,Scholarship holder,3.228869e-01,Forte,1.163099,Baixa multicolinearidade
6,Course_Animação e Design Multimédia,5.418770e-02,Fraca,2.146684,Baixa multicolinearidade
7,Course_Design de Comunicação,7.438562e-03,Inútil,2.115650,Baixa multicolinearidade
8,Course_Enfermagem,9.130095e-02,Fraca,4.972251,Baixa multicolinearidade
9,Course_Enfermagem Veterinária,3.335161e-05,Inútil,2.554063,Baixa multicolinearidade


As transformações aplicadas resultaram em uma melhoria nos parâmetros das variáveis, tornando o conjunto de dados mais adequado para a modelagem. No entanto, ainda observamos que algumas variáveis acadêmicas apresentam sobreajuste, enquanto outras variáveis, especialmente relacionadas ao modo de inscrição (`Application mode`), possuem alta multicolinearidade, o que pode impactar negativamente a estabilidade do modelo.

Diante dessa análise, adotaremos uma abordagem estratégica para a construção do modelo. Vamos criar dois conjuntos de dados:

- **Conjunto completo**: incluirá todas as variáveis, permitindo uma avaliação abrangente do impacto de cada uma no desempenho do modelo.

- **Conjunto otimizado**: conterá apenas as variáveis com os melhores parâmetros, ou seja, aquelas que possuem um IV significativo e baixos níveis de multicolinearidade. Esse conjunto visa reduzir o risco de sobreajuste e melhorar a interpretabilidade do modelo.

Com essa estratégia, poderemos comparar o desempenho dos dois modelos e identificar a configuração mais eficiente e robusta para a predição da variável `Target`.

## | Criando Bases de Dados para Modelagem

In [18]:
# Loop para selecionar apenas as features que atendem às condições
selected_features = []
for index, row in resultado.iterrows():
    if row['IV_Classification'] in ['Sobreajuste', 'Forte', 'Média', 'Fraca'] and row['VIF_Classification'] in ['Baixa multicolinearidade', 'Moderada multicolinearidade']:
        selected_features.append(row['Feature'])

# Criando subconjunto com as features selecionadas
df_sub = df_transformado[selected_features].copy()

# Adicionando Target na base
df_sub['Target'] = df_transformado['Target'].copy()

# Visualizando subconjunto
display(df_sub.head(5))
display(df_sub.shape)

# Visualizando base total
display(df_transformado.head(5))
display(df_transformado.shape)

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Course_Animação e Design Multimédia,Course_Enfermagem,Course_Engenharia Informática,Course_Gestão (atendimento noturno),Course_Serviço Social,Target
0,0.677640,0.855619,0.718660,0.505317,1,1,0,0,0,0,1,0
1,-0.066414,0.304727,-1.445110,-1.735681,1,0,0,0,0,0,1,1
2,-1.554522,-1.898842,-1.445110,-1.735681,1,0,0,0,0,0,0,1
3,1.049667,0.493027,1.079288,0.575895,1,1,0,1,0,0,0,0
4,0.677640,0.558010,0.718660,0.596330,1,0,0,1,0,0,0,0


(76518, 12)

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Target,Course_Animação e Design Multimédia,Course_Design de Comunicação,Course_Enfermagem,...,Application mode_Mudança de curso,Application mode_Mudança de instituição/curso,"Application mode_Portaria n.º 533-A/99, alínea b2) (Plano Diferente)","Application mode_Portaria nº 533-A/99, item b3 (Outra Instituição)",Application mode_Portaria nº 612/93,Application mode_Portaria nº 854-B/99,Application mode_Titulares de diploma de especialização tecnológica,Application mode_Titulares de diplomas de ciclo curto,Application mode_Titulares de outros cursos superiores,Application mode_Transferência
0,0.677640,0.855619,0.718660,0.505317,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.066414,0.304727,-1.445110,-1.735681,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.554522,-1.898842,-1.445110,-1.735681,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.049667,0.493027,1.079288,0.575895,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.677640,0.558010,0.718660,0.596330,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


(76518, 39)

Para melhorar a padronização das colunas, vamos aplicar uma limpeza no nome das colunas para remover parênteses e espaços. Além disso, para facilitar a análise vamos dividir os dados em dois subconjuntos:

- **Base de desenvolvimento**: utilizada para treinar e ajustar o modelo.

- **Base de validação**: usada para avaliar o desempenho do modelo em dados não vistos, garantindo uma melhor generalização.

In [19]:
# Renomeando colunas
df_transformado.columns = [col.replace(' ', '_').replace('(', '').replace(')', '') for col in df_transformado.columns]
df_sub.columns = [col.replace(' ', '_').replace('(', '').replace(')', '') for col in df_sub.columns]

# Criando base de validação com 5% dos dados de cada base
df_valid_t = df_transformado.sample(frac=.05, random_state=412)
df_valid_s = df_sub.sample(frac=.05, random_state=412)

# Criando base de desenvolvimento com 95% dos dados restentantes
df_desen_t = df_transformado.drop(df_valid_t.index)
df_desen_s = df_sub.drop(df_valid_s.index)

print(f'Dimensões da base de desenvolvimento dos dados transformados: {df_desen_t.shape}\nDimensões da base de validação dos dados transformados: {df_valid_t.shape}')
print(f'Dimensões da base de desenvolvimento do subconjunto de dados selecionados: {df_desen_s.shape}\nDimensões da base de validação do subconjunto de dados selecionados: {df_valid_s.shape}')

Dimensões da base de desenvolvimento dos dados transformados: (72692, 39)
Dimensões da base de validação dos dados transformados: (3826, 39)
Dimensões da base de desenvolvimento do subconjunto de dados selecionados: (72692, 12)
Dimensões da base de validação do subconjunto de dados selecionados: (3826, 12)


Para concluir este notebook, vamos salvar os conjuntos de dados preparados no formato **Feather (.ftr)**, que é eficiente e preserva a estrutura dos dados. O formato garante alta performance tanto na leitura quanto na escrita, tornando-o ideal para o armazenamento das bases que serão usadas na próxima etapa de modelagem.

Com isso, as bases de dados estarão prontas para serem carregadas de maneira rápida e eficiente no próximo notebook.

In [24]:
df_valid_t.to_feather('../data/processed/valid_t.ftr')
df_valid_s.to_feather('../data/processed/valid_s.ftr')
df_desen_t.to_feather('../data/processed/data_t.ftr')
df_desen_s.to_feather('../data/processed/data_s.ftr')